<a href="https://colab.research.google.com/github/akshaygrao77/DeepLearning-Assignment2/blob/main/Part_C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np 
import argparse
import time
import imutils
import os
from google.colab import files

In [ ]:
!pip install pytube

     |████████████████████████████████| 56 kB 4.3 MB/s 


In [ ]:
from google.colab.patches import cv2_imshow
from pytube import YouTube

In [ ]:
#Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pytube import YouTube

In [ ]:
!pip install moviepy
from moviepy.editor import *

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)3325952/45929032 bytes (7.2%)6881280/45929032 bytes (15.0%)10469376/45929032 bytes (22.8%)14049280/45929032 bytes (30.6%)17416192/45929032 bytes (37.9%)20725760/45929032 bytes (45.1%)24174592/45929032 bytes (52.6%)27525120/45929032 bytes (59.9%)30728192/45929032 bytes (66.9%)34152448/45929032 bytes (74.4%)37601280/45929032 bytes (81.9%)40878080/45929032 bytes (89.0%)

In [ ]:
ROOT_DIR = os.getcwd()

In [ ]:

#Load yolo
def load_yolo():
	net = cv2.dnn.readNet("drive/MyDrive/DL-assignment2/yolov3-spp.weights", "drive/MyDrive/DL-assignment2/yolov3-spp.cfg")
	classes = []
	with open("drive/MyDrive/DL-assignment2/coco.names", "r") as f:
		classes = [line.strip() for line in f.readlines()] 
	
	output_layers = [layer_name for layer_name in net.getUnconnectedOutLayersNames()]
	colors = np.random.uniform(0, 255, size=(len(classes), 3))
	return net, classes, colors, output_layers

def load_image(img_path):
	# image loading
	img = cv2.imread(img_path)
	img = cv2.resize(img, None, fx=0.4, fy=0.4)
	height, width, channels = img.shape
	return img, height, width, channels


def display_blob(blob):
	'''
		Three images each for RED, GREEN, BLUE channel
	'''
	for b in blob:
		for n, imgb in enumerate(b):
			cv2.imshow(str(n), imgb)

def detect_objects(img, net, outputLayers):			
	blob = cv2.dnn.blobFromImage(img, scalefactor=0.00392, size=(320, 320), mean=(0, 0, 0), swapRB=True, crop=False)
	net.setInput(blob)
	outputs = net.forward(outputLayers)
	return blob, outputs

def get_box_dimensions(outputs, height, width):
	boxes = []
	confs = []
	class_ids = []
	for output in outputs:
		for detect in output:
			scores = detect[5:]
			class_id = np.argmax(scores)
			conf = scores[class_id]
			if conf > 0.3:
				center_x = int(detect[0] * width)
				center_y = int(detect[1] * height)
				w = int(detect[2] * width)
				h = int(detect[3] * height)
				x = int(center_x - w/2)
				y = int(center_y - h / 2)
				boxes.append([x, y, w, h])
				confs.append(float(conf))
				class_ids.append(class_id)
	return boxes, confs, class_ids
			
def draw_labels(boxes, confs, colors, class_ids, classes, img): 
	indexes = cv2.dnn.NMSBoxes(boxes, confs, 0.5, 0.4)
	font = cv2.FONT_HERSHEY_PLAIN
	for i in range(len(boxes)):
		if i in indexes:
			x, y, w, h = boxes[i]
			label = str(classes[class_ids[i]])
			color = colors[i]
			cv2.rectangle(img, (x,y), (x+w, y+h), color, 2)
			cv2.putText(img, label, (x, y - 5), font, 1, color, 1)
	# cv2_imshow(img)

def image_detect(img_path): 
	model, classes, colors, output_layers = load_yolo()
	image, height, width, channels = load_image(img_path)
	blob, outputs = detect_objects(image, model, output_layers)
	boxes, confs, class_ids = get_box_dimensions(outputs, height, width)
	draw_labels(boxes, confs, colors, class_ids, classes, image)
	while True:
		key = cv2.waitKey(1)
		if key == 27:
			break

# def start_object_detection_in_video(video_path,output_file_name):
#   model, classes, colors, output_layers = load_yolo()
#   cap = cv2.VideoCapture(video_path)
#   writer = None
#   processed_frame_count = 0

#   # try to determine the total number of frames in the video file
#   try:
#     prop = cv2.cv.CV_CAP_PROP_FRAME_COUNT if imutils.is_cv2() \
#     else cv2.CAP_PROP_FRAME_COUNT
#     total = int(cap.get(prop))
#     print("[INFO] {} total frames in video".format(total))
#   # an error occurred while trying to determine the total
#   # number of frames in the video file
#   except:
#     print("[INFO] could not determine # of frames in video")
#     print("[INFO] no approx. completion time can be provided")
#     total = -1

#   while True:
#     processed_frame_count += 1
#     if(processed_frame_count % 25 == 0):
#       print("Processed frames:"+str(processed_frame_count))
    
#     (grabbed, frame) = cap.read()
#     if not grabbed:
#       break
#     height, width, channels = frame.shape
#     blob, outputs = detect_objects(frame, model, output_layers)
#     boxes, confs, class_ids = get_box_dimensions(outputs, height, width)
#     draw_labels(boxes, confs, colors, class_ids, classes, frame)
#     # check if the video writer is None
#     if writer is None:
# 		# initialize our video writer
#       fourcc = cv2.VideoWriter_fourcc(*"MJPG")
#       # outvid = os.path.join(ROOT_DIR, "out.mp4")
#       writer = cv2.VideoWriter(output_file_name, fourcc, 30,
#                                (frame.shape[1], frame.shape[0]), True)
      
# 	  # write the output frame to disk
#     writer.write(frame)
#   # release the file pointers
#   print("[INFO] cleaning up...")
#   writer.release()


In [ ]:
def start_object_detection_in_video(video_path,output_file_name,original_fps):
  model, classes, colors, output_layers = load_yolo()
  cap = cv2.VideoCapture(video_path)
  writers = None
  processed_frame_count = 0
  part_output_file_name_list = []
  part_output_file_name = ""
  total_frames = -1
  frame_per_part = original_fps * 10
  
	# try to determine the total number of frames in the video file
  try:
    prop = cv2.cv.CV_CAP_PROP_FRAME_COUNT if imutils.is_cv2() \
    else cv2.CAP_PROP_FRAME_COUNT
    total_frames = int(cap.get(prop))
    writers = [None]*((total_frames // frame_per_part)+1)
    print("[INFO] {} total frames in video".format(total_frames))
  # an error occurred while trying to determine the total
  # number of frames in the video file
  except:
    print("[INFO] could not determine # of frames in video")
    print("[INFO] no approx. completion time can be provided")
    total_frames = -1

  part_count = 0
  while True:
    processed_frame_count += 1
    if(processed_frame_count % 25 == 0):
      print("Processed frames:"+str(processed_frame_count))
    
    (grabbed, frame) = cap.read()
    if not grabbed:
      break
    height, width, channels = frame.shape
    blob, outputs = detect_objects(frame, model, output_layers)
    boxes, confs, class_ids = get_box_dimensions(outputs, height, width)
    draw_labels(boxes, confs, colors, class_ids, classes, frame)
    part_output_file_name = str(output_file_name)+str("_part")+str(part_count)+".avi"
    # check if the video writer is None
    if writers[part_count] is None:
		# initialize our video writer
      fourcc = cv2.VideoWriter_fourcc(*"MJPG")
      # outvid = os.path.join(ROOT_DIR, "out.mp4")
      writers[part_count] = cv2.VideoWriter(part_output_file_name, fourcc, original_fps,
                               (frame.shape[1], frame.shape[0]), True)
      
	  # write the output frame to disk
    writers[part_count].write(frame)
    if(processed_frame_count % frame_per_part == 0):
        writers[part_count].release()
        print("Wrote video:"+str(part_output_file_name))
        # files.download(part_output_file_name)
        part_output_file_name_list.append(part_output_file_name)
        part_count += 1
  
  if(total_frames % frame_per_part != 0):
    writers[part_count].release()
    print("Wrote video:"+str(part_output_file_name))
    # files.download(part_output_file_name)
    part_output_file_name_list.append(part_output_file_name)
	
  return part_output_file_name_list,original_fps

In [ ]:
def merge_files(part_output_file_name_list,target_file_name,original_fps):
  clips = [VideoFileClip(each_part_file_name) for each_part_file_name in part_output_file_name_list]
  # concatenating both the clips
  final = concatenate_videoclips(clips)
  #writing the video into a file / saving the combined video
  final.write_videofile(target_file_name, fps=original_fps,codec="libx264")


In [ ]:
def start_object_detection_on_youtube_video(video_link,download_video_file_name,output_file_name,interval=None):
  video = YouTube(video_link)
  video_file_name = video.title
  print("Video title: "+str(video.title))
  resolutions_order = ["1080p","720p","480p","360p"]
  res_found = False
  count = 0
  fps = 30
  while(not(res_found)):
    download = video.streams.filter(res=resolutions_order[count])
    if(len(download) >0):
      print("Video format which will be downloaded:"+str(download.first()))
      fps = download.first().fps
      print("Frame per second:"+str(fps))
      res_found = True
      download.first().download(filename=str(download_video_file_name)+".mp4" )
    count += 1
  if(not(interval is None)):
    download_video_file_name = cut_and_save_video(interval,download_video_file_name,fps)
  return start_object_detection_in_video(str(download_video_file_name)+".mp4",output_file_name,fps)

In [ ]:
def get_stream_filters_of_video(video_link):
  video = YouTube(video_link)
  return video.streams.all()

In [ ]:
def cut_and_save_video(interval,input_file,original_fps):
  clip_path = input_file+"_clip"
  clip = VideoFileClip(input_file+".mp4") 
      
  clip = clip.subclip(interval[0],interval[1])
  clip.write_videofile(clip_path+".mp4", fps=original_fps,codec="libx264")
  return clip_path

In [ ]:
# https://www.youtube.com/watch?v=c0FtiZUO9Kg   botswana
# https://www.youtube.com/watch?v=_EjWEwjak_Q wildfire
# https://www.youtube.com/watch?v=gs415cKPASA 10second person clip
# https://www.youtube.com/watch?v=EP90lkEwctw giraffe with zerbra
input_file = "giraffe_zebra"
target_file_name = "giraffe_zebra_detection"
part_output_file_name_list,original_fps = start_object_detection_on_youtube_video(video_link="https://www.youtube.com/watch?v=EP90lkEwctw",download_video_file_name=input_file,output_file_name=target_file_name,interval=(5,70))
print("Out files:"+str(part_output_file_name_list))
merge_files(part_output_file_name_list,target_file_name+".avi",original_fps)
files.download(target_file_name+".avi")
# start_object_detection_in_video("drive/MyDrive/DL-assignment2/bird.mp4",output_file_name="bird_detection.avi")

Video title: Giraffe and Zebra and Wildebeest - Amazing & Beautiful Interaction
Video format which will be downloaded:<Stream: itag="137" mime_type="video/mp4" res="1080p" fps="25fps" vcodec="avc1.640028" progressive="False" type="video">
Frame per second:25
[MoviePy] >>>> Building video giraffe_zebra_clip.mp4
[MoviePy] Writing video giraffe_zebra_clip.mp4


100%|█████████▉| 1625/1626 [02:54<00:00,  9.30it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: giraffe_zebra_clip.mp4 

[INFO] 1625 total frames in video
Processed frames:25
Processed frames:50
Processed frames:75
Processed frames:100
Processed frames:125
Processed frames:150
Processed frames:175
Processed frames:200
Processed frames:225
Processed frames:250
Wrote video:giraffe_zebra_detection_part0.avi
Processed frames:275
Processed frames:300
Processed frames:325
Processed frames:350
Processed frames:375
Processed frames:400
Processed frames:425
Processed frames:450
Processed frames:475
Processed frames:500
Wrote video:giraffe_zebra_detection_part1.avi
Processed frames:525
Processed frames:550
Processed frames:575
Processed frames:600
Processed frames:625
Processed frames:650
Processed frames:675
Processed frames:700
Processed frames:725
Processed frames:750
Wrote video:giraffe_zebra_detection_part2.avi
Processed frames:775
Processed frames:800
Processed frames:825
Processed frames:850
Processed frames:875
Processed frames:900
Proces

100%|█████████▉| 1625/1626 [03:17<00:00,  8.22it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: giraffe_zebra_detection.avi 



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>